# Simulation of Myc distribution

11.8.2021 (8.9.2022) Sören Doose

In [ ]:
import sys
from pathlib import Path
import re
import logging
import itertools

%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy import stats
import boost_histogram as bh

import locan as lc

In [ ]:
lc.show_versions(dependencies=False)

In [ ]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

In [ ]:
rng = np.random.default_rng(seed=1)

## Synthetic data

SMLM data are simulated that resemble a homogeneous distribution of dyes where each dye provides several localizations (following a normal distribution with a std equal to the localization precision).
This sort of clustered data is described by a Thomas point process if n_localizations_per_dye is taken from a Poisson distribution. For better resembling dSTORM data we take n_localizations_per_dye from a geometric distribution.

In [ ]:
# endogeneous Myc, saturated labeling
n_localizations_per_dye = 11
min_localizations_per_dye = 1
localization_density = 1.3e-3
intensity_dyes = localization_density / n_localizations_per_dye
localization_precision = 12
# region = lc.Rectangle((0, 0), 4000, 4000, 0)
region = lc.Rectangle((0, 0), 40_000, 40_000, 0)

print("Intensity_dyes:", intensity_dyes)

In [ ]:
%%time
locdata = lc.simulate_dstorm(parent_intensity=intensity_dyes, region=region, cluster_mu=n_localizations_per_dye, min_points=min_localizations_per_dye, cluster_std=localization_precision, seed=rng)

In [ ]:
locdata.print_summary()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 12))
lc.render_2d_mpl(locdata, bin_size=20);

## Cluster

Identify cluster by running a HDBSCAN or DBSCAN clustering routine.

In [ ]:
%%time
noise, clust = lc.cluster_dbscan(locdata, eps=20, min_samples=3)  

In [ ]:
n_clustered_loc = np.sum([ref.properties['localization_count'] for ref in clust.references])
print(f"Number of clusters: {clust.properties['localization_count']}")
print(f"Number of noise localizations: {noise.properties['localization_count']}")
print(f"Number of clustered localizations: {n_clustered_loc}")
print(f"Ratio cluster to noise localizations: {n_clustered_loc / (n_clustered_loc + noise.properties['localization_count']):.3}")

In [ ]:
clust.data.localization_count.describe()

In [ ]:
clust.update_convex_hulls_in_references()
clust.data.fillna(0, inplace=True)

### Number of localizations per cluster

In [ ]:
clust_prop_nloc = lc.LocalizationProperty(loc_property='localization_count').compute(clust)

In [ ]:
clust_prop_nloc.hist(bins=100);

In [ ]:
clust_prop_nloc.hist(bins=np.linspace(0, 100, 20));

In [ ]:
clust_prop_nloc.distribution_statistics.parameter_dict()

### Ratio of cluster above a certain size

In [ ]:
threshold = 100
large_spot_ratio = len(clust.data.localization_count[clust.data.localization_count > threshold]) / clust.properties['localization_count']
round(large_spot_ratio, ndigits=5)

### Localizations in cluster above a certain size

In [ ]:
threshold = 1100

large_spot_localization_ratio = clust.data.localization_count[clust.data.localization_count > threshold].sum() / clust.data.localization_count.sum()
round(large_spot_localization_ratio, ndigits=5)

### Ripley's H function computation

In [ ]:
%%time
n_points = 100
radii = np.linspace(0, 500, 50)

# data
# rhf = lc.RipleysHFunction(radii=radii, region_measure=region.region_measure).compute(locdata=locdata)

subset = lc.random_subset(locdata, n_points=n_points, seed=rng)
rhf = lc.RipleysHFunction(radii=radii, region_measure=region.region_measure).compute(locdata, other_locdata=subset)


# randomized
locdata_csr = lc.randomize(locdata, hull_region=locdata.region, seed=rng)
# rhf_csr = lc.RipleysHFunction(radii=radii, region_measure=region.region_measure).compute(locdata=locdata_csr)

subset_csr = lc.random_subset(locdata_csr, n_points=n_points, seed=rng)
rhf_csr = lc.RipleysHFunction(radii=radii, region_measure=region.region_measure).compute(locdata_csr, other_locdata=subset_csr)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15,12))
lc.render_2d_mpl(locdata_csr, bin_size=20);

In [ ]:
rhf.plot()
rhf_csr.plot()